In [ ]:
import pandas as pd
import numpy as np

## Load Data

In [ ]:
IDS_MAPPING_FN = "./data/IDS_mapping.csv"
DIABETIC_FN = "./data/diabetic_data.csv"

In [ ]:
# read files
mapping = pd.read_csv(IDS_MAPPING_FN, header=None)
df = pd.read_csv(DIABETIC_FN)

In [ ]:
MEDIC_COLUMNS = df.columns[23:46].tolist()
PREVIOUS_HOSPITAL_ENCOUNTERS = ["number_outpatient", "number_inpatient", "number_emergency"]

Create dictionary of code-value mappings of `admission_type_id`, `discharge_disposition_id`, and `admission_source_id` using the mapping provided in the data-folder, and map integer values to string values for readability. 

In [ ]:
admission_type_dict = {}
discharge_disposition_dict = {}
admission_source_dict = {}

list1 = []
for i, j in zip(mapping[0].values, mapping[1].values):
  if len(str(i))>3:
    feature_name = i
  elif len(str(i))!=3:
    if feature_name == 'admission_type_id':
      admission_type_dict[int(i)] = j
    elif feature_name == 'discharge_disposition_id':
      discharge_disposition_dict[int(i)] = j
    elif feature_name == 'admission_source_id':
      admission_source_dict[int(i)] = j


df['admission_type'] = df['admission_type_id'].map(admission_type_dict).astype("O")
df['discharge_disposition'] = df['discharge_disposition_id'].map(discharge_disposition_dict).astype("O")
df['admission_source'] = df['admission_source_id'].map(admission_source_dict).astype("O")

In [ ]:
print(f"Number of unique encounters: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

## Preprocessing

Some patients have many encounters (up to 40).

We only keep the first observation for each unique patient to treat them as i.i.d random variables.

We filter to only keep observations with `admission_type` $\in$ [Emergency, Urgent, Elective].

In [ ]:
df = df.groupby("patient_nbr").agg('first').reset_index()
df = df[df['admission_type'].isin(['Emergency', 'Urgent', 'Elective'])]
print(f"Number of unique encounters after only keeping first encounter for each patient and filtering by admission_type : {df.shape[0]}")

#### Feature Engineering: Diabetic Information

To select features as predictor variables we are only interested in certain information about the patients, but not the exact values. Thus we aggregate the information of sets of columns into new columns:


* Blood Glucose Tests

    * `max_glu_serum_flag`: Whether a max glucose serum test was done at the hospital : `max_glu_serum`


    * `A1C_flag`: Whether an AC1 test was performed to monitor blood glucose levels : `AC1result`

    
* Diabetic Medication Information: 

    * `change_dosage`: Whether there was any change ("Up" or "Down") in the diabetic prescription dosages as a result of the hospital visit.
        * *The column `change` indicates if there was a change in diabetic medications (either dosage or generic name). So if `change` is marked as changed but `change_dosage` is none, then there must have been a change in the generic name, i.e. the chemical name of a medicine.*


    * `change_medicine`: Whether there was prescribed any new diabetic medication i.e. a change in the medicament as a result of the hospital visit.


    * `num_diabetic_prescriptions`: How many diabetic prescriptions the patient ongoingly had at the time of hospital visit - a count of entries that are $\in \{\text{"Steady"}, \text{"Up"}, \text{"Down"}\}$ in `MEDIC_COLUMNS`


* Admitted in the hospital within the previous year

    * `prev_year_hospital`: Whether the patient had any admissions in the hospital during the past year. 

* Health Insurance / Coverage

    * `blue_cross`: Patient has private insurance
    * `medicaid`: Patient has medicaid
    * `medicare`: Patient has medicare
    * `self_payed`: Patient payed up front

* Readmitted
    * `readmitted_flag`: Whether the patient was readmitted or not within 30 days based on the columns `readmitted`


In [ ]:
#### Blood glucose Tests ####
df['max_glu_serum_flag'] = df['max_glu_serum'].notnull().astype(int) # Max Glucose Serum test flag
df['A1C_flag'] = df['A1Cresult'].notnull().astype(int) # AC1 test flag

#### Diabetic Medication Information ####
# Change in dosage if any of the diabetics prescriptions has entries "Up" or "Down"
df['change_dosage'] = df[MEDIC_COLUMNS].isin(['Up', 'Down']).any(axis=1).astype(int)
# Check if each entry is in the set ['Up', 'Down', 'Steady'] and sum all True entries for each row
df['num_diabetic_prescriptions'] = df[MEDIC_COLUMNS].apply(lambda col: np.isin(col, ['Up', 'Down', 'Steady'])).sum(axis=1).astype(int)
# Change in medicine is assumed to be the case when original change column = 1 but change in dosage = 0
df['change_medicine'] = np.where((df['change'] == 'Ch') & (df['change_dosage'] == 0), 1, 0)

#### Hospital Encounters during preceding year ####
df['prev_year_hospital'] = (df[PREVIOUS_HOSPITAL_ENCOUNTERS] > 0).any(axis=1).astype(int)

#### Insurance Billing ####
df['blue_cross'] = np.where(df["payer_code"]=="BC", 1, 0)
df['medicare'] = np.where(df["payer_code"]=="MC", 1, 0)
df['medicaid'] = np.where(df["payer_code"]=="MD", 1, 0)
df['self_pay'] = np.where(df["payer_code"]=="SP", 1, 0)

#### Readmitted ####
# y_i
df['readmitted_flag'] = np.where(df['readmitted']=='<30', 1, 0) # Readmitted flag

## Define variables

In [ ]:
FEATURES = ["time_in_hospital", "admission_type_id", "num_lab_procedures", "num_procedures", 
            "num_medications", 'prev_year_hospital', "number_diagnoses", 'max_glu_serum_flag', 
            'A1C_flag', 'change_dosage', 'change_medicine', 'num_diabetic_prescriptions', "blue_cross", "medicaid", "medicare","self_pay"]
PROTECTED_FEATURES = ['age', 'race', 'gender']
PATIENTS = ["patient_nbr"]  
TARGET = ["readmitted_flag"]

In [ ]:
df = df[PATIENTS + FEATURES + PROTECTED_FEATURES + TARGET]

In [ ]:
df